In [1]:
import pint
import xarray as xr
from dask.distributed import Client
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import (
    FunctionalGroupOptimizeNoTransport,
    GeneticAlgorithm,
    GeneticAlgorithmParameters,
    NoTransportCostFunction,
    Observation,
    Parameter,
    constraint,
)

Load forcing.


In [2]:
time_start, time_end = "1998-01-01", "2022-01-01"

bats_data = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cmems.zarr", engine="zarr")
bats_data["T"].attrs["units"] = StandardUnitsLabels.temperature.units
bats_data = bats_data.sel(time=slice(time_start, time_end))
_ = bats_data.load()

cafe_npp = xr.open_dataset("../notebooks/1_data_processing/1_1_Forcing/bats_cafe.zarr", engine="zarr")
cafe_npp = cafe_npp.sel(time=slice(time_start, time_end))
cafe_npp = cafe_npp.dropna("time")
cafe_npp = cafe_npp.resample(time="D").interpolate("linear")
cafe_npp.lat.attrs = {"units": "degrees_north", "long_name": "latitude", "axis": "Y"}
cafe_npp.lon.attrs = {"units": "degrees_east", "long_name": "longitude", "axis": "X"}
cafe_npp = cafe_npp.rename({"lat": "latitude", "lon": "longitude"})
_ = cafe_npp.load()

In [ ]:
bats_data

Load observations.

First I multiply the observations by the average epipelagic layer depth (150m) to have a biomass in m2 rather than m3.


In [4]:
def update_layer(data: xr.DataArray, epipelagic_size):
    data = data.pint.quantify() * epipelagic_size
    layer_attrs = data["layer"].attrs
    data = data.assign_coords({"layer": [1]})
    data["layer"].attrs = layer_attrs
    return data


epipelagic_size = 150 * pint.application_registry("meter")

obs_bats = xr.open_dataset("../../seapopym-data/data/zooplankton_product/Bats_zooplankton.nc")
obs_bats = update_layer(obs_bats, epipelagic_size)
obs_bats = Observation(obs_bats)
observations = [obs_bats]

Create structure for SeapoPym simulation.


In [ ]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=bats_data["T"], resolution=1 / 12),
    primary_production=ForcingUnit(forcing=cafe_npp["CAFE"], resolution=1 / 12),
)

Setup the cost function.


In [6]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=1,
        night_layer=1,
        tr_rate=Parameter("D1N1_tr_rate", -1, 0),
        tr_max=Parameter("D1N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.05, 0.8),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=2,
        night_layer=1,
        tr_rate=Parameter("D2N1_tr_rate", -1, 0),
        tr_max=Parameter("D2N1_tr_max", 0, 50),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", 0, 1),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 0, 100),
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.05, 0.8),
    ),
]

In [7]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=observations,
)

Set the genetic algorithm meta parameters.


In [8]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    ETA=1, INDPB=0.1, CXPB=0.5, MUTPB=0.2, NGEN=20, POP_SIZE=20, cost_function_weight=(-1,)
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [9]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [10]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

And watch the magic on the Dask dashboard :


In [ ]:
genetic_algo.client

And execute the process.


In [ ]:
viewer = genetic_algo.optimize()

Finaly here is the result :


In [ ]:
viewer.box_plot(4)

In [ ]:
viewer.parallel_coordinates()